In [1]:
import numpy

In [4]:
# This function takes a square matrix and sets the new value of (row, col) to the old value of (col, col) - (row, col).
# So, when you feed it the matrix Utility(row | col), it gives you back the matrix Improvement(col,row | col)
def improve(m):
    blank = numpy.zeros(shape=(m.shape[0],m.shape[1]))
    for i in range(0,m.shape[0]):
        for j in range(0,m.shape[1]):
            blank[i][j] = m[j][j]-m[i][j]
    return blank

In [5]:
# This function takes a square matrix and sets the new value of (row, col) to the old value of (row, col) - (col, row).
# So, when you feed it the matrix Improvement(col, row | col), it gives you back the matrix News(col, row)
def newsify(m):
    blank = numpy.zeros(shape=m.shape)
    for row in range(0,m.shape[0]):
        for col in range(0,m.shape[1]):
            blank[row][col] = m[row][col] - m[col][row]
    return blank

In [6]:
# Here, I generate the matrix News(col, row) from the user-provided desires and probability matrices
News = newsify(improve(des.dot(probs)))

In [7]:
# Later, I need to know how many acts there are.  So I store the number of rows in News as "num_acts"
num_acts = News.shape[0]

In [8]:
# This function generates a list of ordered triples, where the first member of the triple is the column of News(col, row), 
# the second member is the row of News(col,row), and the third member is the value at (row, col).
# Because News(col, row) = - News(row, col), I only look at the upper triangle
def pairs(m):
    blist = []
    for row in range(0,m.shape[0]):
        for col in range(row+1,m.shape[1]):
            blist.append((col, row, m[row][col]))
    return blist

In [9]:
# Running the function "pairs" on the matrix "News" to get the list "pairs"
pairs = pairs(News)

In [ ]:
# This function replaces numbers with the user-provided names for the acts.
def name_acts(l):
    first = input('What name would you like to give to the act in the first row of the desire matrix?')
    key_dict = { 0: first }
    count = 1
    for num in range(1,num_acts):
        next = input('What name would you like to give to the act in the next row?')
        key_dict[count] = next
        count += 1
    blist = []
    for row in range(0,len(l)):
        blist.append((key_dict[l[row][0]], key_dict[l[row][1]], l[row][2]))
    return blist

In [12]:
# Init_list is the result of replacing each row number with the user-provided names for acts
init_list = name_acts(pairs)

In [ ]:
# The 'positivise' function makes all strengths positive by swapping the position of options if the strength is negative and 
# replacing strength with its absolute value
def positivise(l):
    blist = []
    for row in range(0,len(l)):
        if l[row][2] >0:
            blist.append((l[row][0], l[row][1],l[row][2]))
        elif l[row][2]<0:
            blist.append((l[row][1], l[row][0],abs(l[row][2])))
        elif l[row][2]==0:
            blist.append((l[row][0],l[row][1],l[row][2]))
    return blist

In [ ]:
# The list 'positive_pairs' gives the 'positivised' init_list
positive_pairs = positivise(init_list)

In [15]:
# This function checks to see whether there are ties in the "strength" of reasons 
def check_for_ties(l):
    answer = 0
    for i in range(0,len(l)):
        for j in range(i+1,len(l)):
            if abs(l[i][2])==abs(l[j][2]):
                answer = 1
                return answer
    return answer

In [16]:
# This function checks for ties.  If it finds them, then it prompts the user to supply a 'tie-breaking' preference ordering,
# which it then uses to break the ties.  It does this in the following way: if X > Y is tied with Z > W, then 'X > Y' is placed
# above 'Z > W' iff X comes before Y in the 'tie-breaking' preferences.
# If X > Y is tied with X > Z, then X > Y is placed above X > Z iff Z comes before Y in the 'tie-breaking' preferences.
# If X = Y is tied with Z = W, then X = Y is placed above Z = W iff X comes before Z in the 'tie-breaking' preferences.
# And if X = Y is tied with X = Z, then X = Y comes first iff Y is preferred to Z.
# This way of 'breaking ties' will count as 'rationalisable', in the terms from TCDTGTMTN.
def break_ties(l):
    if check_for_ties(l)==0:
        l.sort(key=lambda x:abs(x[2]),reverse=True)
    if check_for_ties(l)==1:
        print()
        print('Because there are ties in the strengths of your reasons, this decision requires the use of the tie-breaking algorithm.')
        print('You should enter some arbitrary preference between options which we will use to break ties.')
        print('Be sure to use the same names for the acts that you entered above.')
        print()
        first = input("What is your most preferred option?")
        my_dict = { first : 0 }
        count=1
        for num in range(1,num_acts):
            next = input("What is your next most preferred option?")
            my_dict[next] = count
            count+=1
        l.sort(key=lambda x: (-x[2], my_dict[x[0]], -my_dict[x[1]]) if x[2] > 0 else (-x[2], my_dict[x[0]], my_dict[x[1]]))
        #l.sort(key=lambda x:abs(x[2]),reverse=True)
    return l

In [ ]:
# Running the "break_ties" function on the list "positive_pairs"
break_ties(positive_pairs)

In [20]:
#This function displays the preferences and strengths 
def pref_view_init(l):
    for entry in l:
        up = entry[0]
        down=entry[1]
        if entry[2]>0:
            print(up+'  >  '+down+'    ['+str(entry[2])+']')
        if entry[2]==0:
            print(up+'  =  '+down+'    ['+str(entry[2])+']')

In [21]:
#This function displays just the preferences
def pref_view(l):
    for entry in l:
        up = entry[0]
        down=entry[1]
        if entry[2]>0:
            print(up+'  >  '+down)
        if entry[2]==0:
            print(up+'  =  '+down)

In [ ]:
# Here are the tuples, ordered by the strength of the reason you have to prefer option1 to option2
print()
print("Here is the list of preferences which you have pro tanto reason to hold, ordered by the strength of the reason you have to hold them.")
print()
pref_view_init(positive_pairs)
print()

In [23]:
# With this function, we chain together any pairs where the 1st option matches the 2nd or the 2nd matches the first.
# Thus, X > Y and Y > Z leads to a pair X > Z
# And X > Y and Z = X leads to a pair Z > Y
# And X = Y and Y = Z leads to X = Z
def chain(l):
    blist = []
    for row2 in range(0,len(l)):
        for row1 in range(0,row2):
            if l[row1][2]!=0 or l[row2][2]!=0:
                if l[row1][0]==l[row2][1] and not (l[row2][0],l[row1][1],max(l[row1][2],l[row2][2])) in blist:
                    blist.append((l[row2][0],l[row1][1],max(l[row1][2],l[row2][2])))
                if l[row1][1]==l[row2][0] and not (l[row1][0],l[row2][1],max(l[row1][2],l[row2][2])) in blist:
                    blist.append((l[row1][0],l[row2][1],max(l[row1][2],l[row2][2])))
        blist.append(l[row2])
    return blist

In [24]:
# This function checks a list of preferences to see whether they form any cycles.
# It does this by running the "chain" function as many times as their are acts, effectively tracing out every possible path 
# through the options.  If any of these paths take you back to the option you started with, then the function 
# returns the answer "1".  Otherwise, it returns "0".
# Additionally, we check any indifferences in the list to see whether they conflict with a strict preference in the list
# For instance, X = Y conflicts with X > Y and Y > X
def check_for_cycles(l):
    blist = l
    answer = 0
    for num in range(0,num_acts+1):
        blist=chain(blist)
        for row in blist:
            if row[0]==row[1]:
                answer = 1
                return answer
            if row[2]==0:
                for row2 in blist:
                    if row2[2]>0:
                        if (row[0]==row2[0] and row[1]==row2[1]) or (row[0]==row2[1] and row[1]==row2[0]):
                            answer = 1
                            return answer                    
    return answer

In [25]:
# This function decides which strict preferences are kept by adding the strict preferences
# from the sorted list iff doing so does not lead to any cycles.  We ignore indifferences.
def kept_pairs(l):
    keep = []
    for row in range(0,len(l)):
        if l[row][2]!=0 and check_for_cycles(keep + [l[row]])==0:
            keep.append(l[row])
        if l[row][2]==0 and check_for_cycles(keep + [l[row]])==0 and check_for_cycles(keep + [[l[row][1],l[row][0],0]])==0:
            keep.append(l[row])
    return keep 

In [26]:
# This list called "keepers" is just the preferences which are not struck (i.e., those which do not lead to cycles
# when combined with the ones above them)
keepers = kept_pairs(positive_pairs)

In [ ]:
print()
print("We start at the top of this list, and work our way down, striking any preferences which lead to cycles when")
print("combined with the (unstruck) preferences above them.  When we reach the bottom, the following preferences are")
print("left unstruck:")
print()
pref_view(keepers)
print()

In [28]:
# This function returns the options which are at the top of the preference ordering.  It scans through every pair of distinct
# strict preferences in a list and returns options which are strictly preferred to some option and not strictly *dis*preferred
# to any other option
# It additionally scans through the indifferences to see whether there are 
def top(l):
    tops=[]
    for i in l:
        dispreferred=0
        if i[2]>0:
            for j in l:
                if i!=j and i[2] != 0 and j[2]!=0 and j[1]==i[0]:
                    dispreferred=1
            if dispreferred==0:
                tops.append(i[0])
        if i[2]==0:
            for j in l:
                if i!=j and j[2] != 0 and (j[1]==i[0] or j[1]==i[1]):
                    dispreferred=1
            if dispreferred==0:
                tops.append(i[0])
                tops.append(i[1])
    return tops

In [29]:
# This function removes any duplicates from a list.
def remove_duplicates(l):
    blist = [] 
    [blist.append(x) for x in l if x not in blist]
    return blist

In [30]:
# This function displays every item in a list on its own line
def line_by_line(l):
    for item in l:
        print(item)

In [ ]:
print()
print('Therefore, the following options are permissible:')
print()
line_by_line(remove_duplicates(top(keepers)))